In [1]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.pipeline import make_pipeline

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2022/12/23 23:55:10 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-remote-sm/1', creation_time=1671819910753, experiment_id='1', last_update_time=1671819910753, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [3]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=143)
    mlflow.log_params

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
        )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model", registered_model_name="sk-learn-rf-reg-model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 143} 6.754617267244934


Successfully registered model 'sk-learn-rf-reg-model'.
2022/12/24 00:04:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-rf-reg-model, version 1
Created version '1' of model 'sk-learn-rf-reg-model'.


In [8]:
from mlflow.tracking import MlflowClient

In [17]:
MLFLOW_TRACKING_URI = "https://127.0.0.1:5000"
RUN_ID = "34df38edeebf46b390642bae174fc435"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

path = client.download_artifacts(run_id=RUN_ID, path="model/model.pkl")

In [18]:
with open(path, 'rb') as f_out:
    model_imported = pickle.load(f_out)

In [19]:
ride = {
    "PULocationID": 10,
    "DOLocationID": 55,
    "trip_distance": 40
}

print(model_imported.predict(ride))

[44.90644052]
